In [1]:
from transfer_learning_publication.models import ModelFactory
from transfer_learning_publication.data import CaravanDataSource, LSHDataModule
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore
from transfer_learning_publication.models import ModelFactory
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
import joblib

## Transform data and sink to disk

In [2]:
ts_features = ["temperature_2m_max",
            "temperature_2m_min",
            "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
            "potential_evaporation_sum_ERA5_LAND",
            "temperature_2m_mean",
            "total_precipitation_sum",
            "streamflow",
            "streamflow_was_filled",
            "sin_day_of_year",
            "cos_day_of_year"]

static_features = ["p_mean",
            "area",
            "ele_mt_sav",
            "high_prec_dur",
            "frac_snow",
            "high_prec_freq",
            "slp_dg_sav",
            "cly_pc_sav",
            "aridity_ERA5_LAND",
            "aridity_FAO_PM",]

stages = ["train", "val", "test"]

In [3]:
# ts_preprocessing_pipelines = (
#     PipelineBuilder(group_identifier="gauge_id")
#     .add_per_basin(Log(), columns=["streamflow"])
#     .add_global(ZScore(), columns=["temperature_2m_max",
#             "temperature_2m_min",
#             "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
#             "potential_evaporation_sum_ERA5_LAND",
#             "temperature_2m_mean",
#             "total_precipitation_sum",
#             "streamflow"])
#     .build()
# )

# static_preprocessing_pipelines = (
#     PipelineBuilder(group_identifier="gauge_id")
#     .add_global(ZScore(), columns=static_features)
#     .build()
# )

# for stage in stages:
#     caravan = CaravanDataSource(f"/Users/cooper/Desktop/CARAVAN_CLEAN/{stage}", region="tajikkyrgyz")
#     gauge_ids = caravan.list_gauge_ids()
#     ts_data = caravan.get_timeseries(
#         gauge_ids[:3], 
#         columns=ts_features
#     )
#     static_data = caravan.get_static_attributes(
#         gauge_ids[:3], 
#         columns=static_features
#     )

#     if stage == "train":
#         ts_transformed = ts_preprocessing_pipelines.fit_transform(ts_data.collect())
#         static_transformed = static_preprocessing_pipelines.fit_transform(static_data.collect())
#     else:
#         ts_transformed = ts_preprocessing_pipelines.transform(ts_data.collect())
#         static_transformed = static_preprocessing_pipelines.transform(static_data.collect())

#     caravan.write_timeseries(ts_transformed, f"/Users/cooper/Desktop/first-test/{stage}", overwrite=False)
#     caravan.write_static_attributes(static_transformed, f"/Users/cooper/Desktop/first-test/{stage}", overwrite=False)

#     print(f"Completed {stage} stage")

In [4]:
# # Dump the fitted pipelines
# joblib.dump(ts_preprocessing_pipelines, "/Users/cooper/Desktop/first-test/ts_pipeline.joblib")
# joblib.dump(static_preprocessing_pipelines, "/Users/cooper/Desktop/first-test/static_pipeline.joblib")

## Create datamodule and instantiate model for training

In [5]:
config_path = "/Users/cooper/Desktop/transfer-learning-publication/configs/first_run.yaml"

In [6]:
datamodule = LSHDataModule(config_path=config_path)

model_factory = ModelFactory()
model = model_factory.create_from_config(config_path=config_path)

In [7]:
trainer = pl.Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    logger=True,
    callbacks=[
        ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1),
        EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=True)
    ],
    enable_progress_bar=True,
    log_every_n_steps=1
)

trainer.fit(model=model, datamodule=datamodule)  

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | criterion | MSELoss   | 0      | train
1 | rev_in    | RevIN     | 2      | train
2 | model     | TiDEModel | 178 K  | train
------------------------------------------------
178 K     Trainable params
0         Non-tr

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/Users/cooper/Desktop/transfer-learning-publication/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 13/13 [00:04<00:00,  3.16it/s, v_num=1, val_loss=0.0445]

...

Epoch 3: 100%|██████████| 13/13 [00:00<00:00, 21.02it/s, v_num=1, val_loss=0.0325]

Metric val_loss improved by 0.003 >= min_delta = 0.0. New best score: 0.032


Epoch 4: 100%|██████████| 13/13 [00:00<00:00, 23.75it/s, v_num=1, val_loss=0.0305]

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.031
`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 13/13 [00:00<00:00, 22.28it/s, v_num=1, val_loss=0.0305]
